In [13]:
import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
from datetime import datetime
import matplotlib.pyplot as plt

In [14]:
def average_deviation(data):
    data = np.array(data)
    mean_value = np.mean(data)
    res = 0
    for x in data:
        res += abs(x-mean_value)
    res = res/np.size(data)
    return '{:.2f}'.format(res)

In [15]:
# process data produced by curl
# download 1K.bin
def curl_data():
    f = open('curldata.txt')
    speed = [[], []]
    time = [[], []]

    for line in f:
        if 'iperf server' in line:
            curser = int(line.split(' ')[2])
            continue
        if 'EEST' in line:
            date = line.split(' EEST')[0].split(':0')[0]
            date = date.split('T')[0] + ' '+ date.split('T')[1]
            time[curser-1].append(date)
        if '100 1024k' in line:
            avg_speed = line.split(' ')
            while '' in avg_speed:
                avg_speed.remove('')
            avg_speed = avg_speed[6]+'bps'
            speed[curser-1].append(avg_speed)
    out = [float(x.split('M')[0]) for x in speed[0]]
    mean = str('{:.2f}'.format(np.mean(out)))+'Mbps'
    med = str(np.median(out))+'Mbps'
    mininum = str(min(out))+'Mbps'
    maximum = str(max(out))+'Mbps'
    avgd = str(average_deviation(out))+'Mbps'
    out = speed[0] + [mean, med, mininum, maximum, avgd]
    time[0] = time[0] + ['Mean', 'Median', 'Minimum', 'Maximum', 'Average deviation']
    return time[0], out
    
time_curl, speed_curl = curl_data()

In [16]:
# process data produced by curl
# download 500M.bin
def curl_data2():
    f = open('curldata2.txt')
    speed = []
    time = []
    temp = 0
    for line in f:
        if 'EEST' in line:
            date = line.split(' EEST')[0].split(':0')[0]
            date = date.split('T')[0] + ' '+ date.split('T')[1]
            time.append(date)
            continue
        if 'speed_download' in line:
            temp = 1
            continue
        if temp == 1 and ',' in line:
            avg_speed = float(line.split(', ')[0].split('\n')[0])
            speed.append(avg_speed)
            temp = 0

    out = [float('{:.2f}'.format(x/1000000)) for x in speed]
    mean = str('{:.2f}'.format(np.mean(out)))+' Mbps'
    med = str('{:.2f}'.format(np.median(out)))+' Mbps'
    mininum = str(min(out))+' Mbps'
    maximum = str(max(out))+' Mbps'
    avgd = str(average_deviation(out))+' Mbps'
    speed = [str(x)+' Mbps' for x in out] 
    out = speed + [mean, med, mininum, maximum, avgd]
    time = time + ['Mean', 'Median', 'Minimum', 'Maximum', 'Average deviation']
    return time, out

time_curl2, speed_curl2 = curl_data2()


In [17]:
class record:
    def __init__(self):
        self.time = []
        self.normal = []
        self.reverse = []
        self.complete = 0

def iperf_data():
    f = open('iperfdata.txt')
    records = [[], []]
    mode = 0
    
    for line in f:
        if 'Reverse mode' in line:
            mode = 1
            continue
        if 'iperf server' in line:
            curser = int(line.split(' ')[2].split(':')[0])
            cur_record = record()
            continue
        if 'EEST' in line:
            cur_record.time = line.split(' EEST')[0]
            continue
        if 'receiver' in line:
            # take the bitrate at the receiver side
            bitrate = line.split('  ')[5]
            if mode == 0:
                cur_record.normal = bitrate
                mode = 1
            else:
                cur_record.reverse = bitrate
                mode = 0
        if 'error' in line:
            if mode == 0:
                cur_record.normal = 'error'
                mode = 1
            else:
                cur_record.reverse = 'error'
                mode = 0
        if cur_record.normal != [] and cur_record.reverse != []:
            records[curser-1].append(cur_record)
            cur_record = record()
            
    time = [x.time for x in records[0]]
    speed_up = [x.normal for x in records[0]]
    speed_dn = [x.reverse for x in records[0]]
    #calculation
    speed_up_value = []
    speed_dn_value = []
    for i in range(0,24):
        if speed_up[i] != 'error':
            speed_up[i] = speed_up[i].split('Gbits')[0]+'Gbps'
            speed_up_value.append(float(speed_up[i].split(' Gbps')[0]))
        if speed_dn[i] != 'error':
            speed_dn[i] = speed_dn[i].split('Gbits')[0]+'Gbps'
            speed_dn_value.append(float(speed_dn[i].split(' Gbps')[0]))    
    
    mean = str('{:.2f}'.format(np.mean(speed_up_value)))+' Gbps'
    med = str('{:.2f}'.format(np.median(speed_up_value)))+' Gbps'
    mininum = str(min(speed_up_value))+' Gbps'
    maximum = str(max(speed_up_value))+' Gbps'
    avgd = str(average_deviation(speed_up_value))+'Gbps'
    speed_up=speed_up+[mean, med, mininum, maximum, avgd]

    mean = str('{:.2f}'.format(np.mean(speed_dn_value)))+' Gbps'
    med = str("{:.2f}".format(np.median(speed_dn_value)))+' Gbps'
    mininum = str(min(speed_dn_value))+' Gbps'
    maximum = str(max(speed_dn_value))+' Gbps'
    avgd = str(average_deviation(speed_dn_value))+'Gbps'
    speed_dn=speed_dn+[mean, med, mininum, maximum, avgd]
    time = time + ['mean', 'median', 'minimum', 'maximum', 'average deviation']
    
    return time, speed_up, speed_dn
    
time_iperf, iperf_ul, iperf_dl = iperf_data()


In [18]:
def st_data():
    f = open('stdata.txt')
    time = []
    speed_up = []
    speed_dn = []
    for line in f:
        if 'time' in line:
            date = line.split('time: ')[1].split('\n')[0]
            time.append(date)
        elif 'upload' in line:
            up = line.split('upload: ')[1].split('\n')[0]
            speed_up.append(up)
        elif 'download' in line:
            speed_dn.append(line.split('download: ')[1].split('\n')[0])
        else:
            continue
    #calculation
    speed_up_value = [float(x.split(' Mbps')[0]) for x in speed_up]
    speed_dn_value = [float(x.split(' Mbps')[0]) for x in speed_dn]
    
    mean = str('{:.2f}'.format(np.mean(speed_up_value)))+' Mbps'
    med = str('{:.2f}'.format(np.median(speed_up_value)))+' Mbps'
    mininum = str(min(speed_up_value))+' Mbps'
    maximum = str(max(speed_up_value))+' Mbps'
    avgd = str(average_deviation(speed_up_value))+' Mbps'
    speed_up=speed_up+[mean, med, mininum, maximum, avgd]

    mean = str('{:.2f}'.format(np.mean(speed_dn_value)))+' Mbps'
    med = str("{:.2f}".format(np.median(speed_dn_value)))+' Mbps'
    mininum = str(min(speed_dn_value))+' Mbps'
    maximum = str(max(speed_dn_value))+' Mbps'
    avgd = str(average_deviation(speed_dn_value))+' Mbps'
    speed_dn=speed_dn+[mean, med, mininum, maximum, avgd]
    
    time = time + ['Mean', 'Median', 'Minimum', 'Maximum', 'Average deviation']
    
    return time, speed_up, speed_dn
            
st_time, st_up, st_dn = st_data()  


In [19]:
from IPython.display import display
def draw_table():
    table_data = {
    'Time':time_curl, 
    'HTTP speed':speed_curl, 
    # 'HTTP speed':speed_curl2, 
    #'time_iperf':time_2, 
    'Iperf upload':iperf_ul, 
    'Iperf download':iperf_dl,
    'ST upload':st_up,
    'ST download':st_dn,
    'ST time':st_time
    }
    df = pd.DataFrame(table_data)
    display(df)
    
draw_table()

,Time,HTTP speed,Iperf upload,Iperf download,ST upload,ST download,ST time
0,2021-10-19 18:32,386.07 Mbps,5.30 Gbps,7.37 Gbps,363.17 Mbps,300.50 Mbps,10-19 19:05
1,2021-10-19 19:32,574.88 Mbps,error,9.06 Gbps,309.15 Mbps,358.17 Mbps,10-19 19:06
2,2021-10-19 20:32,322.84 Mbps,6.16 Gbps,6.99 Gbps,325.48 Mbps,375.42 Mbps,10-19 19:07
3,2021-10-19 21:32,548.42 Mbps,5.20 Gbps,9.09 Gbps,373.99 Mbps,424.00 Mbps,10-19 22:51
4,2021-10-19 22:32,375.3 Mbps,5.29 Gbps,9.21 Gbps,381 Mbps,450 Mbps,10-19 22:53
5,2021-10-19 23:32,316.98 Mbps,error,error,271.69 Mbps,387.37 Mbps,10-19 22:57
6,2021-10-20 00:32,417.43 Mbps,error,9.28 Gbps,349.19 Mbps,386.20 Mbps,10-20 15:32
7,2021-10-20 01:32,450.03 Mbps,5.08 Gbps,7.20 Gbps,290.20 Mbps,341.89 Mbps,10-20 15:33
8,2021-10-20 02:32,385.79 Mbps,5.46 Gbps,9.31 Gbps,260.89 Mbps,348.29 Mbps,10-20 15:34
9,2021-10-20 03:32,557.16 Mbps,5.34 Gbps,7.25 Gbps,289.08 Mbps,368.50 Mbps,10-20 16:01
